# IsoNet
------
This will be used to create the model for predicting precipitable stable water isotopes. Only creates and exports the model and trains it. Is not used to predict.

Table of Contents:
1. [Importing Data](#importing-data)
2. [Data Preprocessing](#data-preprocessing)
3. [Model Creation](#model-creation)
4. [Model Training](#model-training)
5. [Model Export](#model-export)

In [29]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import datetime
import matplotlib.pyplot as plt

# Tensorflow and keras libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, InputLayer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

## Importing Data
Importing the data, and then splitting it into features and labels

In [30]:
# Load in the data, separate the features and labels
data = pd.read_csv(r'Isoscape_Data.csv')
features = data.drop(['H2avg', 'dex', 'O18Avg', 'Station'], axis=1)
labels = data['O18Avg']

# Adjusting the date time to be int values where Janurary 1st, of year 1 is 1
features['Date'] = pd.to_datetime(features['Date'])
features['Date'] = features['Date'].map(datetime.datetime.toordinal)

numFeatures = features.shape[1]

## Data Preprocessing
Preprocessing the data, including:
* Convert into numpy arrays
* Split into training and testing sets
* Convert into tensorflow Datasets
* Windowing the data (Skipping for now, will do later) 
* Standardizing the data

In [31]:
# Split the data into training and testing sets
splitIndex = int(0.8 * len(features))
xTrain = features[:splitIndex]
xTest = features[splitIndex:]
yTrain = labels[:splitIndex]
yTest = labels[splitIndex:]

# Scale the data
scaler = StandardScaler()
xTrain = scaler.fit_transform(xTrain)
xTest = scaler.transform(xTest)

# Convert the data into numpy arrays
xTrain = np.array(xTrain)
xTest = np.array(xTest)
yTrain = np.array(yTrain)
yTest = np.array(yTest)

# Convert traindata into tensor dataset
trainData = tf.data.Dataset.from_tensor_slices((xTrain, yTrain))

## Model Creation
Creating the model, including:
* Creating the model architecture
* Compiling the model

Model architecture:
* Input layer
* 1 LSTM layer
* 2 Dense layers
* Output layer

In [32]:
numNeuorns = 64
model = Sequential()
model.add(InputLayer(input_shape=(numFeatures, 1)))
model.add(LSTM(numNeuorns))
model.add(Dense(numNeuorns, activation='relu'))
model.add(Dense(numNeuorns, activation='relu'))
model.add(Dense(1))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

## Model Training
Training and evaluating the model

In [33]:
es = EarlyStopping(monitor='mean_absolute_error', patience=10)

batchSize = 64
model.fit(
    trainData.batch(batchSize),
    batch_size=batchSize,
    epochs=500
)

Epoch 1/500


23/23 [==============================] - 3s 6ms/step - loss: 384.6304 - root_mean_squared_error: 19.6120 - mean_absolute_error: 17.6983
Epoch 2/500
23/23 [==============================] - 0s 7ms/step - loss: 337.9477 - root_mean_squared_error: 18.3834 - mean_absolute_error: 16.0832
Epoch 3/500
23/23 [==============================] - 0s 7ms/step - loss: 145.9774 - root_mean_squared_error: 12.0821 - mean_absolute_error: 9.7469
Epoch 4/500
23/23 [==============================] - 0s 7ms/step - loss: 55.8985 - root_mean_squared_error: 7.4765 - mean_absolute_error: 5.9636
Epoch 5/500
23/23 [==============================] - 0s 7ms/step - loss: 44.0383 - root_mean_squared_error: 6.6361 - mean_absolute_error: 5.2119
Epoch 6/500
23/23 [==============================] - 0s 6ms/step - loss: 35.4736 - root_mean_squared_error: 5.9560 - mean_absolute_error: 4.7521
Epoch 7/500
23/23 [==============================] - 0s 6ms/step - loss: 32.8875 - root_mean_squared_error: 5.7348 - mean_absolute_err

## Model Export

In [34]:
# Printing how well the model did on the test data
print(model.evaluate(xTest, yTest))

12/12 [==============================] - 1s 3ms/step - loss: 13.7925 - root_mean_squared_error: 3.7138 - mean_absolute_error: 2.8308
[13.792458534240723, 3.713819980621338, 2.8308489322662354]


In [35]:
# Create new pd DataFrame with the predictions and actual values including the features
predictions = model.predict(xTest)
df = features.loc[splitIndex:]
df['Predictions'] = predictions
df['Actual'] = yTest

# Export the data to a csv file
df.to_csv('results_test.csv', index=False)

12/12 [==============================] - 0s 3ms/step


/tmp/ipykernel_5119/3509207634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predictions'] = predictions
/tmp/ipykernel_5119/3509207634.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Actual'] = yTest


In [36]:
# Save the model
model.save('printModel.keras')

In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 64)                16896     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 25281 (98.75 KB)
Trainable params: 25281 (98.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
